In [ ]:
samples = [(0, 36188), (1, 5174), (2, 1312), (3, 968), (4, 962), (5, 628), (6, 562)]

labels = ['Normal', 'Bot', 'Infilteration', 'DDos', 'DoS', 'Web', 'Brute-force']

# Calculate the total number of samples
total_samples = sum([count for _, count in samples])
# Calculate the proportion of each class
proportions = [(cls, count / total_samples) for cls, count in samples]

print(proportions)

In [2]:
import torch
from torch import nn
 
# NLLLoss+LogSoftmax
# logsoftmax=log(softmax(x))
m = nn.LogSoftmax(dim=1) #横向计算
loss = nn.NLLLoss()
# 3行5列的输入，即3个样本各包含5个特征，每个样本通过softmax产生5个输出
input = torch.randn(3, 5, requires_grad=True)
#这里的104就是标签，即input中的第一行的识别出来的标签为1，第二行的标签为0
target = torch.tensor([1, 0, 4]) 
# NLL将取输出矩阵中第0行的第1列、第1行的第0列、第2行的第4列加负号求和
output = loss(m(input), target)


In [3]:
input

tensor([[ 0.5614,  0.2378,  0.4337, -0.0815, -0.8311],
        [-0.9018,  1.1236,  1.2969, -0.4175,  0.6018],
        [ 1.7519,  1.1706,  1.1448, -0.9843,  2.2610]], requires_grad=True)

In [4]:
m(input)

tensor([[-1.2173, -1.5408, -1.3450, -1.8601, -2.6098],
        [-3.1661, -1.1406, -0.9673, -2.6818, -1.6625],
        [-1.3436, -1.9248, -1.9507, -4.0798, -0.8345]],
       grad_fn=<LogSoftmaxBackward0>)

In [19]:
import torch.nn as nn
import torch
from thop import profile
from thop import clever_format
from pytorch_model_summary import summary

In [20]:
class GCE(nn.Module):
    def __init__(self, in_features, num_classes, dev='cpu'):
        super(GCE, self).__init__()
        self.in_features = in_features
        self.num_classes = num_classes
        self.embedding = nn.Embedding(num_classes, in_features)
        self.dev = dev

    def forward(self, x, label):
        embeddings = self.embedding(torch.tensor(range(self.num_classes), device=self.dev))
        cosine = F.linear(F.normalize(x), F.normalize(embeddings))
        one_hot = torch.zeros(cosine.size(), device=self.dev)
        one_hot.scatter_(1, label.view(-1, 1).long(), 1)

        softmax_value = F.log_softmax(cosine, dim=1)
        softmax_loss = one_hot * softmax_value
        softmax_loss = - torch.mean(torch.sum(softmax_loss, dim=1))

        return softmax_loss


class CoV(nn.Module):
    def __init__(self, in_dim):
        super(CoV, self).__init__()

        self.Conditional_gamma = nn.Sequential(
            nn.Linear(in_dim, in_dim),
            nn.ReLU(),
            nn.LayerNorm([in_dim]),
        )
        self.Conditional_beta = nn.Sequential(
            nn.Linear(in_dim, in_dim),
            nn.ReLU(),
            nn.LayerNorm([in_dim]),
        )
        self.act = nn.ReLU()

    def forward(self, x, context):
        gamma = self.Conditional_gamma(context)
        beta = self.Conditional_beta(context)

        out = torch.multiply(x, gamma + 1)
        out = torch.add(out, beta)
        out = self.act(out)
        return out

In [21]:
GCE = GCE(in_features=80,num_classes=7)
CoV = CoV(80)

In [22]:
input_tensor = torch.rand(1024,80)
model_summary=summary(GCE, input_tensor)

TypeError: forward() missing 1 required positional argument: 'label'

In [14]:
class CICIDS_Transformer(nn.Module):
    def __init__(self, in_feature=80,out_features=7,d_model=80,nhead=2, dim_feedforward=512, num_layers=5, dropout=0.1):
        super().__init__()


        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=dim_feedforward,
            dropout=dropout
        )
        self.transformer_encoder = nn.TransformerEncoder(
            encoder_layer,
            num_layers=num_layers
        )
        self.fc = nn.Linear(in_feature, out_features)
        self.d_model = d_model

    def forward(self, x):
        x = self.transformer_encoder(x)
        # x = self.fc1(x)
        x = self.fc(x)
        return x

In [16]:
model = CICIDS_Transformer(in_feature=122, out_features=5, d_model=122)
input_tensor = torch.rand(1024,122)
model_summary=summary(model, input_tensor)
print(model_summary)

----------------------------------------------------------------------------
           Layer (type)        Output Shape         Param #     Tr. Param #
   TransformerEncoder-1         [1024, 122]         930,370         930,370
               Linear-2           [1024, 5]             615             615
Total params: 930,985
Trainable params: 930,985
Non-trainable params: 0
----------------------------------------------------------------------------


In [18]:
model = nn.Linear(80,7, bias=True)
input_tensor = torch.rand(1024,80)
model_summary=summary(model, input_tensor)
print(model_summary)

-----------------------------------------------------------------------
      Layer (type)        Output Shape         Param #     Tr. Param #
          Linear-1           [1024, 7]             567             567
Total params: 567
Trainable params: 567
Non-trainable params: 0
-----------------------------------------------------------------------


In [5]:
# model = CICIDS_Transformer(in_feature=122, out_features=5, d_model=122)
model= nn.Linear(80,7,bias=True)

In [6]:
input = torch.randn(1024,80)  # 随机生成一个输入张量，这个尺寸应该与模型输入的尺寸相匹配
flops, params = profile(model, inputs=(input,))

# 将结果转换为更易于阅读的格式
flops, params = clever_format([flops, params], '%.3f')

[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.


In [7]:

print(f"运算量：{flops}, 参数量：{params}")

运算量：573.440K, 参数量：0.000B
